In [1]:
import numpy as np
import pickle as pkl
import torch
from scipy.spatial.distance import cosine, euclidean
import pandas as pd
import pprint as pp

In [2]:
#import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
%matplotlib inline

In [5]:
import bz2
import _pickle as cPickle

# Load any compressed pickle file
def decompress_pickle(file):
    assert file.endswith("pbz2")
    data = bz2.BZ2File(file, "rb")
    data = cPickle.load(data,encoding="latin1")
    return data

In [6]:
#data

In [7]:
run_no = "1"

In [8]:
is_abs_A = False

In [9]:
cc_start_entity = "t" # s, t

In [10]:
dname_data = "./../../data/wiki"+run_no+"/"

In [11]:
fname_data = dname_data+"dict_data_item_subj_v4_hidden_run"+run_no+".pkl.p2.pbz2"

In [12]:
#data_dict = pkl.load(open(fname_data,"rb"))
data_dict = decompress_pickle(fname_data)

In [13]:
X_ts = data_dict["data_hidden"]['X_item_subj']
X_sb = data_dict["data_hidden"]['X_subj_bow']
X_tb = data_dict["data_hidden"]['X_item_bow']
    
X_bs = X_sb.T
X_bt = X_tb.T
X_st = X_ts.T

In [14]:
dict_map_wiki_e = {}
dict_map_wiki_e["p"] = "b"
dict_map_wiki_e["r"] = "s"
dict_map_wiki_e["d"] = "t"

dict_map_wiki_x = {}
dict_map_wiki_x["X_pr"] = "X_bs"
dict_map_wiki_x["X_pd"] = "X_bt"
dict_map_wiki_x["X_rd"] = "X_st"

In [15]:
G = {
    "b":["X_bs","X_bt"],\
    "s":["X_bs","X_st"],\
    "t":["X_st","X_bt"]}

X_data = {
    "X_bs":X_bs, \
    "X_bt":X_bt, \
    "X_st":X_st
    }

X_meta = {
    "X_bs":["b","s"],
    "X_bt":["b","t"],
    "X_st":["s","t"]}

X_dtype = {
    "X_bs":"real", \
    "X_bt":"real", \
    "X_st":"real"
}
    
dict_num_clusters = {"b":3,"s":3,"t":3}

In [16]:
#out

In [17]:
dname_out = "./../../out_clust/wiki"+run_no+"/version_2/"

In [18]:
fname_clust_labels = dname_out+"dict_c_clust_labels.pkl"
fname_A = dname_out+"dict_A.pkl"
fname_X_prime = dname_out+"dict_recons_X.pkl"
fname_Y_prime = dname_out+"dict_recons_Y.pkl"
fname_I_ortho = dname_out+"dict_I_ortho.pkl"
fname_U = dname_out+"dict_U.pkl"
fname_mu = dname_out+"dict_mu.pkl"

In [19]:
dict_clust_labels = pkl.load(open(fname_clust_labels,"rb"))
dict_A = pkl.load(open(fname_A,"rb"))
dict_X_prime = pkl.load(open(fname_X_prime,"rb"))
dict_Y_prime = pkl.load(open(fname_Y_prime,"rb"))
dict_I_ortho = pkl.load(open(fname_I_ortho,"rb"))
dict_U = pkl.load(open(fname_U,"rb"))
dict_mu = pkl.load(open(fname_mu,"rb"))

In [20]:
#sanity check - start

In [21]:
dict_A.keys()

dict_keys(['X_ts', 'X_tb', 'X_sb'])

In [22]:
dict_A["X_ts"]

array([[ 5.7952213 , -4.0037937 , -6.3939776 ],
       [-3.9246604 , -1.0156132 ,  0.764707  ],
       [-0.86962116,  7.875843  ,  7.9190187 ]], dtype=float32)

In [23]:
np.round(dict_A["X_ts"])

array([[ 6., -4., -6.],
       [-4., -1.,  1.],
       [-1.,  8.,  8.]], dtype=float32)

In [24]:
A_sr = np.abs(dict_A["X_ts"].T)
scaler1 = StandardScaler()
A_sr = scaler1.fit_transform(A_sr)
A_temp = np.zeros(A_sr.shape)
for i in np.arange(A_temp.shape[0]):
    j = np.argmax(A_sr[i,:])
    A_temp[i,j] = 1
A_temp    

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [25]:
np.round(np.dot(dict_I_ortho["t"].T, dict_I_ortho["t"]))

array([[ 1.,  0.,  0.],
       [ 0.,  1., -0.],
       [ 0., -0.,  1.]], dtype=float32)

In [26]:
#sanity check - end

In [27]:
dict_clust_labels

{'b': array([0, 2, 0, ..., 2, 2, 0], dtype=int32),
 's': array([1, 1, 0, ..., 1, 1, 1], dtype=int32),
 't': array([1, 1, 1, ..., 1, 1, 1], dtype=int32)}

In [28]:
def get_cluster_num_idx(dict_clust_labels,e_id,num_clusters):
    p_clust_id_list = dict_clust_labels[e_id]
#     p_clust_id_list = []
#     for i in np.arange(U.shape[0]):
#         temp_arr = U[i]
#         p_clust_id_list.append(np.argmax(temp_arr))
    #
    p_clust_id_num_dict = {}
    p_clust_id_idx_dict = {}
    for j in np.arange(num_clusters[e_id]):
        temp_clust_j = np.array(p_clust_id_list) == j
        temp_clust_j_idx = np.arange(p_clust_id_list.shape[0])[temp_clust_j]
        num_p = np.sum(temp_clust_j)
        print("cluster: ",j,", #", num_p)
        p_clust_id_num_dict[j] = num_p
        p_clust_id_idx_dict[j] = temp_clust_j_idx     
    return p_clust_id_num_dict, p_clust_id_idx_dict,p_clust_id_list

In [29]:
dict_entity_cluster_info = {}
for e_id in G.keys():
    print("e_id: ",e_id)
    temp_num, temp_idx, temp_pred_idx = get_cluster_num_idx(dict_clust_labels,e_id,dict_num_clusters)
    dict_entity_cluster_info[e_id] = {"num":temp_num,"idx":temp_idx,"temp_pred_idx":temp_pred_idx}

e_id:  b
cluster:  0 , # 1481
cluster:  1 , # 1445
cluster:  2 , # 957
e_id:  s
cluster:  0 , # 232
cluster:  1 , # 564
cluster:  2 , # 320
e_id:  t
cluster:  0 , # 559
cluster:  1 , # 983
cluster:  2 , # 258


In [30]:
dict_A

{'X_sb': array([[ 88.58934  ,   4.637163 ,  30.972359 ],
        [-54.74947  , -53.494663 , -45.806633 ],
        [ -2.7461557,  67.07067  ,  34.440502 ]], dtype=float32),
 'X_tb': array([[ 81.1142   ,   4.1865892,   2.381117 ],
        [-62.82621  , -48.703434 ,   3.6773312],
        [  3.2362955,  88.90382  , -10.682833 ]], dtype=float32),
 'X_ts': array([[ 5.7952213 , -4.0037937 , -6.3939776 ],
        [-3.9246604 , -1.0156132 ,  0.764707  ],
        [-0.86962116,  7.875843  ,  7.9190187 ]], dtype=float32)}

In [31]:
# dict_p_cluster_asso_chain

In [32]:
#try 2

In [33]:
dict_A.keys()

dict_keys(['X_ts', 'X_tb', 'X_sb'])

In [34]:
num_clusters = dict_num_clusters

dict_t_clust_num = dict_entity_cluster_info["t"]["num"]
dict_s_clust_num = dict_entity_cluster_info["s"]["num"]
dict_b_clust_num = dict_entity_cluster_info["b"]["num"]
#
dict_cc1 = {}
for t_clust_id in np.arange(num_clusters["t"]):
    if dict_t_clust_num[t_clust_id] > 0:
        #ts
        if is_abs_A:
            A_ts_temp = dict_A["X_ts"]
            scaler1 = StandardScaler()
            A_ts_temp = np.abs(scaler1.fit_transform(A_ts_temp))
        else:
            A_ts_temp = dict_A["X_ts"]
            scaler1 = StandardScaler()
            A_ts_temp = scaler1.fit_transform(A_ts_temp)
        s_clust_id_list = list(np.argsort(A_ts_temp[t_clust_id,:]))
        s_clust_id_list.reverse()
        for temp_s_id in s_clust_id_list:
            if dict_s_clust_num[temp_s_id] > 0:
                s_clust_id = temp_s_id
                break
    dict_cc1[t_clust_id] = {"t":t_clust_id, "s":s_clust_id}

dict_cc2 = {}
for t_clust_id in np.arange(num_clusters["t"]):
    if dict_t_clust_num[t_clust_id] > 0:
        #tb
        if is_abs_A:
            A_tb_temp = dict_A["X_tb"]
            scaler1 = StandardScaler()
            A_tb_temp = np.abs(scaler1.fit_transform(A_tb_temp))
        else:
            A_tb_temp = dict_A["X_tb"]
            scaler1 = StandardScaler()
            A_tb_temp = scaler1.fit_transform(A_tb_temp)        
        b_clust_id_list = list(np.argsort(A_tb_temp[t_clust_id,:]))
        b_clust_id_list.reverse()
        for temp_b_id in b_clust_id_list:
            if dict_b_clust_num[temp_b_id] > 0:
                b_clust_id = temp_b_id
                break        
        #bs
        if is_abs_A:
            A_bs_temp = dict_A["X_sb"].T
            scaler1 = StandardScaler()
            A_bs_temp = np.abs(scaler1.fit_transform(A_bs_temp))
        else:
            A_bs_temp = dict_A["X_sb"].T
            scaler1 = StandardScaler()
            A_bs_temp = scaler1.fit_transform(A_bs_temp)        
        s_clust_id_list = list(np.argsort(A_bs_temp[b_clust_id,:]))
        s_clust_id_list.reverse()
        for temp_s_id in s_clust_id_list:
            if dict_s_clust_num[temp_s_id] > 0:
                s_clust_id = temp_s_id
                break
    dict_cc2[t_clust_id] = {"t":t_clust_id,"b":b_clust_id, "s":s_clust_id}        

In [35]:
dict_cc1

{0: {'s': 0, 't': 0}, 1: {'s': 2, 't': 1}, 2: {'s': 1, 't': 2}}

In [36]:
dict_cc2

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 2, 's': 1, 't': 1},
 2: {'b': 1, 's': 2, 't': 2}}

In [37]:
# for temp in dict_cc1.keys():
#     dict_cc1[temp].update(dict_cc2[temp])

In [38]:
# dict_temp_cc1 = dict_cc1

In [39]:
dict_temp_cc_t = {}
for temp_key in dict_cc1.keys():
    temp = {}
    cur_t = dict_cc1[temp_key]["t"]
    cur_s = dict_cc1[temp_key]["s"]
    temp["t"] = cur_t
    temp["s"] = cur_s
    for temp_key_2 in dict_cc2.keys():
        if cur_t == dict_cc2[temp_key_2]["t"]:
            cur_b = dict_cc2[temp_key_2]["b"]
            print("cur_t: ",cur_t,", cur_s: ",cur_s,", cur_b: ",cur_b)
            temp["b"] = cur_b
            dict_temp_cc_t[temp_key] = temp

cur_t:  0 , cur_s:  0 , cur_b:  0
cur_t:  1 , cur_s:  2 , cur_b:  2
cur_t:  2 , cur_s:  1 , cur_b:  1


In [40]:
dict_temp_cc_t

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 2, 's': 2, 't': 1},
 2: {'b': 1, 's': 1, 't': 2}}

In [41]:
dict_A.keys()

dict_keys(['X_ts', 'X_tb', 'X_sb'])

In [42]:
num_clusters = dict_num_clusters

dict_t_clust_num = dict_entity_cluster_info["t"]["num"]
dict_s_clust_num = dict_entity_cluster_info["s"]["num"]
dict_b_clust_num = dict_entity_cluster_info["b"]["num"]
#
dict_cc3 = {}
for s_clust_id in np.arange(num_clusters["s"]):
    if dict_s_clust_num[s_clust_id] > 0:
        #st
        if is_abs_A:
            A_st_temp = dict_A["X_ts"].T
            scaler1 = StandardScaler()
            A_st_temp = np.abs(scaler1.fit_transform(A_st_temp))
        else:
            A_st_temp = dict_A["X_ts"].T
            scaler1 = StandardScaler()
            A_st_temp = scaler1.fit_transform(A_st_temp)
        t_clust_id_list = list(np.argsort(A_st_temp[s_clust_id,:]))
        t_clust_id_list.reverse()
        for temp_t_id in t_clust_id_list:
            if dict_t_clust_num[temp_t_id] > 0:
                t_clust_id = temp_t_id
                break
    dict_cc3[s_clust_id] = {"s":s_clust_id, "t":t_clust_id}

dict_cc4 = {}
for s_clust_id in np.arange(num_clusters["s"]):
    if dict_s_clust_num[s_clust_id] > 0:
        #sb
        if is_abs_A:
            A_sb_temp = dict_A["X_sb"]
            scaler1 = StandardScaler()
            A_sb_temp = np.abs(scaler1.fit_transform(A_sb_temp))
        else:
            A_sb_temp = dict_A["X_sb"]
            scaler1 = StandardScaler()
            A_sb_temp = scaler1.fit_transform(A_sb_temp)        
        b_clust_id_list = list(np.argsort(A_sb_temp[s_clust_id,:]))
        b_clust_id_list.reverse()
        for temp_b_id in b_clust_id_list:
            if dict_b_clust_num[temp_b_id] > 0:
                b_clust_id = temp_b_id
                break
        #bt
        if is_abs_A:
            A_bt_temp = dict_A["X_tb"].T
            scaler1 = StandardScaler()
            A_bt_temp = np.abs(scaler1.fit_transform(A_bt_temp))
        else:
            A_bt_temp = dict_A["X_tb"].T
            scaler1 = StandardScaler()
            A_bt_temp = scaler1.fit_transform(A_bt_temp)        
        t_clust_id_list = list(np.argsort(A_bt_temp[b_clust_id,:]))
        t_clust_id_list.reverse()
        for temp_t_id in t_clust_id_list:
            if dict_t_clust_num[temp_t_id] > 0:
                t_clust_id = temp_t_id
                break                 
    dict_cc4[s_clust_id] = {"t":t_clust_id, "b":b_clust_id, "s":s_clust_id}        

In [43]:
dict_cc3

{0: {'s': 0, 't': 0}, 1: {'s': 1, 't': 2}, 2: {'s': 2, 't': 1}}

In [44]:
dict_cc4

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 0, 's': 1, 't': 0},
 2: {'b': 1, 's': 2, 't': 2}}

In [45]:
# for temp in dict_cc3.keys():
#     dict_cc3[temp].update(dict_cc4[temp])

In [46]:
dict_temp_cc_s = {}
for temp_key in dict_cc3.keys():
    temp = {}
    cur_t = dict_cc3[temp_key]["t"]
    cur_s = dict_cc3[temp_key]["s"]
    temp["t"] = cur_t
    temp["s"] = cur_s
    for temp_key_2 in dict_cc4.keys():
        if cur_s == dict_cc4[temp_key_2]["s"]:
            cur_b = dict_cc4[temp_key_2]["b"]
            temp["b"] = cur_b
            dict_temp_cc_s[temp_key] = temp

In [47]:
dict_temp_cc_s

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 0, 's': 1, 't': 2},
 2: {'b': 1, 's': 2, 't': 1}}

In [48]:
dict_temp_cc_t

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 2, 's': 2, 't': 1},
 2: {'b': 1, 's': 1, 't': 2}}

In [49]:
# dict_temp_cc = {}
# count = 0
# for temp_key in dict_temp_cc1.keys():
#     dict_temp_cc[count] = dict_temp_cc1[temp_key]
#     count+=1
# for temp_key in dict_temp_cc2.keys():
#     dict_temp_cc[count] = dict_temp_cc2[temp_key]
#     count+=1    

In [50]:
if cc_start_entity == "s":
    dict_temp_cc = dict_temp_cc_s
elif cc_start_entity == "t":    
    dict_temp_cc = dict_temp_cc_t
else:
    assert False

In [51]:
dict_cluster_asso_chain = dict_temp_cc

In [52]:
dict_cluster_asso_chain

{0: {'b': 0, 's': 0, 't': 0},
 1: {'b': 2, 's': 2, 't': 1},
 2: {'b': 1, 's': 1, 't': 2}}

In [53]:
#try 2

In [54]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [55]:
def get_dist(X1, X2):
    d_list = np.diag(cosine_distances(X1,X2))
    #d = np.sqrt(np.sum(np.round(d_list,4)))
    d = np.mean(np.round(d_list,4))
    return d 

In [56]:
def get_dist_subspace(X1, X2, k, is_X1_orth=False, is_X2_orth=False):
    k1 = X1.shape[1]
    k2 = X2.shape[1]
    k = min(k1,k2)
    print("k1: ",k1,", k2: ",k2,", min(k1,k2): ",k)
    #
    if not is_X1_orth:
        u, s, vh = np.linalg.svd(X1)
        X1_tilde = u[:,0:k]
    if not is_X2_orth:
        u, s, vh = np.linalg.svd(X2)
        X2_tilde = u[:,0:k]
    #
    M = np.dot(X1_tilde.T, X2_tilde)
    #print("M.shape: ",M.shape)
    u, s, vh = np.linalg.svd(M)
    d = np.sqrt(np.sum(1 - np.round(pow(s,2),4)))
    #d = np.mean(1 - np.round(pow(s,2),2))
    return d    

In [57]:
dict_mu.keys()

dict_keys(['t_X_ts', 't_X_tb', 's_X_ts', 's_X_sb', 'b_X_tb', 'b_X_sb'])

In [58]:
dict_mu.keys()

dict_keys(['t_X_ts', 't_X_tb', 's_X_ts', 's_X_sb', 'b_X_tb', 'b_X_sb'])

In [59]:
mu_tb = dict_mu["t_X_tb"]
mu_sb = dict_mu["s_X_sb"]

In [60]:
#X_rd = X_rd
X_st = X_st
X_ts = X_st.T
#X_pr = X_pr
X_bs = X_bs
X_sb = X_bs.T
#X_pd = X_pd
X_bt = X_bt
X_tb = X_bt.T

In [61]:
dict_Y_prime.keys()

dict_keys(['t_X_ts', 't_X_tb', 's_X_ts', 's_X_sb', 'b_X_tb', 'b_X_sb'])

In [62]:
#X_rd
X_st2 = dict_Y_prime["s_X_ts"]
X_ts2 = dict_Y_prime["t_X_ts"]
#X_pr
X_bs2 = dict_Y_prime["b_X_sb"]
X_sb2 = dict_Y_prime["s_X_sb"]
#X_pd
X_bt2 = dict_Y_prime["b_X_tb"]
X_tb2 = dict_Y_prime["t_X_tb"]

In [63]:
dict_X_prime.keys()

dict_keys(['X_ts', 'X_tb', 'X_sb'])

In [64]:
#X_rd
X_st_prime = dict_X_prime["X_ts"].T
X_ts_prime = dict_X_prime["X_ts"]
#X_pr
X_bs_prime = dict_X_prime["X_sb"].T
X_sb_prime = dict_X_prime["X_sb"]
#X_pd
X_bt_prime = dict_X_prime["X_tb"].T
X_tb_prime = dict_X_prime["X_tb"]

In [65]:
dict_entity_cluster_info.keys()

dict_keys(['b', 's', 't'])

In [66]:
dict_cluster_asso_chain[0].keys()

dict_keys(['t', 's', 'b'])

In [67]:
dict_clust_id_score = {}
for temp_key in dict_cluster_asso_chain.keys():
    dict_cc = dict_cluster_asso_chain[temp_key]
    # 
    b_clust_id = dict_cc["b"]
    s_clust_id = dict_cc["s"]
    t_clust_id = dict_cc["t"]
    #
    b_idx_list = dict_entity_cluster_info["b"]["idx"][b_clust_id]
    s_idx_list = dict_entity_cluster_info["s"]["idx"][s_clust_id]
    t_idx_list = dict_entity_cluster_info["t"]["idx"][t_clust_id]
    #
    X_st_cc = X_st[s_idx_list,:][:,t_idx_list]
    X_ts_cc = X_ts[t_idx_list,:][:,s_idx_list]
    X_bs_cc = X_bs[b_idx_list,:][:,s_idx_list]
    X_sb_cc = X_sb[s_idx_list,:][:,b_idx_list]
    X_bt_cc = X_bt[b_idx_list,:][:,t_idx_list]
    X_tb_cc = X_tb[t_idx_list,:][:,b_idx_list]
#     #
#     X_st_cc = X_st2[s_idx_list,:][:,t_idx_list]
#     X_ts_cc = X_ts2[t_idx_list,:][:,s_idx_list]
#     X_bs_cc = X_bs2[b_idx_list,:][:,s_idx_list]
#     X_sb_cc = X_sb2[s_idx_list,:][:,b_idx_list]
#     X_bt_cc = X_bt2[b_idx_list,:][:,t_idx_list]
#     X_tb_cc = X_tb2[t_idx_list,:][:,b_idx_list]
    #
    X_st_prime_cc = X_st_prime[s_idx_list,:][:,t_idx_list]
    X_ts_prime_cc = X_ts_prime[t_idx_list,:][:,s_idx_list]
    X_bs_prime_cc = X_bs_prime[b_idx_list,:][:,s_idx_list]
    X_sb_prime_cc = X_sb_prime[s_idx_list,:][:,b_idx_list]
    X_bt_prime_cc = X_bt_prime[b_idx_list,:][:,t_idx_list]
    X_tb_prime_cc = X_tb_prime[t_idx_list,:][:,b_idx_list]
    #
    mu_tb_cc = mu_tb[t_idx_list,:]
    mu_sb_cc = mu_sb[s_idx_list,:]
    #
    k = 50
    if cc_start_entity == "s":
        s1 = get_dist(X_st_cc, X_st_prime_cc)
        s21 = get_dist(X_sb_cc, X_sb_prime_cc)
        s22 = get_dist(X_bt_cc, X_bt_prime_cc)
        s2 = s21 + s22 
        s3 = get_dist_subspace(X_sb_cc, X_st_prime_cc, k)
        #s3 = get_dist_subspace(mu_sb_cc, X_st_prime_cc, k)
    elif cc_start_entity == "t":
        s1 = get_dist(X_ts_cc, X_ts_prime_cc)
        s21 = get_dist(X_tb_cc, X_tb_prime_cc)
        s22 = get_dist(X_bs_cc, X_bs_prime_cc)
        s2 = s21 + s22
        s3 = get_dist_subspace(X_tb_cc, X_ts_prime_cc, k)
        #s3 = get_dist_subspace(mu_tb_cc, X_ts_prime_cc, k)  
    #
    alpha = 1.0
    beta = 1.0
    gamma = 1.0
    cur_score = (alpha * s1) - (beta * s2) - (gamma * s3)
    #cur_score = (alpha * s1) + (beta * (1-s2)) + (gamma * (1-s3))
    dict_clust_id_score[temp_key] = cur_score
    print("cur_key: ",temp_key,"| s1: ",np.round(s1,4),", s2: ",np.round(s2,4),", s21: ",np.round(s21,4),", s22: ",np.round(s22,4),", s3: ",np.round(s3,4),", score: ",np.round(cur_score,4))

k1:  1481 , k2:  232 , min(k1,k2):  232
cur_key:  0 | s1:  1.0334 , s2:  1.8937 , s21:  1.0164 , s22:  0.8773 , s3:  11.1676 , score:  -12.0278
k1:  957 , k2:  320 , min(k1,k2):  320
cur_key:  1 | s1:  0.997 , s2:  2.1286 , s21:  1.0614 , s22:  1.0671 , s3:  13.7414 , score:  -14.8729
k1:  1445 , k2:  564 , min(k1,k2):  564
cur_key:  2 | s1:  1.0044 , s2:  1.859 , s21:  0.8722 , s22:  0.9868 , s3:  0.0 , score:  -0.8546


In [68]:
dict_clust_id_score

{0: -12.027804134539407, 1: -14.872909277526958, 2: -0.8546354864408143}

In [69]:
keys_list = list(dict_clust_id_score.keys())
val_list = list(np.array(list(dict_clust_id_score.values())) * -1)
min_score_clust_id = keys_list[np.argmax(val_list)]
print("min_key: ",min_score_clust_id)
print("min score:",dict_clust_id_score[min_score_clust_id])

min_key:  1
min score: -14.872909277526958


In [70]:
keys_list = list(dict_clust_id_score.keys())
val_list = list(dict_clust_id_score.values())
max_score_clust_id = keys_list[np.argmax(val_list)]
print("max_key: ",max_score_clust_id)
print("max score:",dict_clust_id_score[max_score_clust_id])

max_key:  2
max score: -0.8546354864408143


In [71]:
keys_list = list(dict_clust_id_score.keys())
val_list = np.abs(list(dict_clust_id_score.values()))
abs_max_score_clust_id = keys_list[np.argmax(val_list)]
print("abs max_key: ",abs_max_score_clust_id)
print("abs max score:",dict_clust_id_score[abs_max_score_clust_id])

abs max_key:  1
abs max score: -14.872909277526958


In [72]:
def get_overlap_percent(dict_entity_cluster_info,e_id,e_clust_id):
    p_cur_idx = list(dict_entity_cluster_info[e_id]['idx'][e_clust_id])
    if e_id == "t":
        list_name = "list_idx_items_to_hide"
    elif e_id == "s":
        list_name = "list_idx_subj_to_hide"
    elif e_id == "b":        
        #list_name = "list_idx_subj_bow_to_hide"
        list_name = "list_idx_item_bow_to_hide"
    #list_name = "list_idx_item_bow_to_hide" #TODO check if list_idx_subj_bow_to_hide == list_idx_item_bow_to_hide
    p_gt_of_idx = data_dict["data_hidden"][list_name]
    #p_gt_of_idx = data_dict["e_unknown_cluster_idx_perm"][e_id]
    p_overlap_idx = list(set(p_cur_idx).intersection(set(p_gt_of_idx)))
    p_per = np.round((len(p_overlap_idx)/len(p_gt_of_idx))*100.0,2)    
    return p_per

In [73]:
data_dict["data_hidden"]["list_tuples_hidden"][:2]

[(1066, 581), (1061, 581)]

In [74]:
data_dict["data_hidden"].keys()

dict_keys(['list_idx_items_to_hide', 'cluster_labels', 'list_idx_subj_to_hide', 'list_idx_item_bow_to_hide', 'X_item_subj', 'X_item_bow', 'hidden_entity_id', 'X_subj_bow', 'percent_hidden', 'list_tuples_hidden', 'list_idx_subj_bow_to_hide'])

In [75]:
data_dict["metadata"].keys()

dict_keys(['X_row_item_labels', 'item_subj_link_words', 'X_row_items_wikidata_id', 'dict_col_subj_link_words', 'dict_col_idx_prop', 'X_col_subj_wikipedia_page', 'dict_row_item_idx', 'X_col_subj_wikidata_id', 'cluster_names', 'cluster_sizes', 'X_col_subj_labels', 'vectorizer.get_feature_names()', 'dict_row_idx_item', 'X_row_items_wikipedia_page', 'dict_row_items_link_words', 'dict_col_prop_idx'])

In [76]:
dict_cluster_asso_chain.keys()

dict_keys([0, 1, 2])

In [77]:
dict_cluster_asso_chain[0].keys()

dict_keys(['t', 's', 'b'])

In [78]:
dict_result1 = {}
for temp_key in dict_cluster_asso_chain.keys():
    temp_res_dict = {}
    #
    b_clust_id = dict_cluster_asso_chain[temp_key]["b"]
    s_clust_id = dict_cluster_asso_chain[temp_key]["s"]
    t_clust_id = dict_cluster_asso_chain[temp_key]["t"]
    #
    b_clust_size = dict_entity_cluster_info["b"]['num'][b_clust_id]
    s_clust_size = dict_entity_cluster_info["s"]['num'][s_clust_id]
    t_clust_size = dict_entity_cluster_info["t"]['num'][t_clust_id]
    #
    b_overlap_per = get_overlap_percent(dict_entity_cluster_info,"b",b_clust_id)
    s_overlap_per = get_overlap_percent(dict_entity_cluster_info,"s",s_clust_id)
    t_overlap_per = get_overlap_percent(dict_entity_cluster_info,"t",t_clust_id)
    #
    temp_res_dict["b_clust_id"] = b_clust_id
    temp_res_dict["s_clust_id"] = s_clust_id
    temp_res_dict["t_clust_id"] = t_clust_id
    #
    temp_res_dict["b_clust_size"] = b_clust_size
    temp_res_dict["s_clust_size"] = s_clust_size
    temp_res_dict["t_clust_size"] = t_clust_size
    #
    temp_res_dict["b_overlap_per"] = b_overlap_per
    temp_res_dict["s_overlap_per"] = s_overlap_per
    temp_res_dict["t_overlap_per"] = t_overlap_per
    #
    dict_result1[temp_key] = temp_res_dict

In [79]:
df_result1 = pd.DataFrame(dict_result1)
pp.pprint(df_result1)
print("#")
df_result1 = pd.DataFrame(dict_result1).T

                     0       1        2
b_clust_id        0.00    2.00     1.00
b_clust_size   1481.00  957.00  1445.00
b_overlap_per    24.91   54.58    20.51
s_clust_id        0.00    2.00     1.00
s_clust_size    232.00  320.00   564.00
s_overlap_per    12.50   50.00    37.50
t_clust_id        0.00    1.00     2.00
t_clust_size    559.00  983.00   258.00
t_overlap_per     0.00  100.00     0.00
#


In [80]:
pp.pprint(df_result1[["b_overlap_per","s_overlap_per","t_overlap_per"]].T)

                   0       1      2
b_overlap_per  24.91   54.58  20.51
s_overlap_per  12.50   50.00  37.50
t_overlap_per   0.00  100.00   0.00


In [81]:
pp.pprint(df_result1[["b_overlap_per","s_overlap_per","t_overlap_per"]].T)

                   0       1      2
b_overlap_per  24.91   54.58  20.51
s_overlap_per  12.50   50.00  37.50
t_overlap_per   0.00  100.00   0.00


In [82]:
max_score_clust_id,min_score_clust_id

(2, 1)

In [83]:
abs_max_score_clust_id

1

In [84]:
df_result1[["b_overlap_per","s_overlap_per","t_overlap_per"]].sum(axis=1)

0     37.41
1    204.58
2     58.01
dtype: float64